In [27]:
import requests
import csv
import json
import time
from datetime import datetime, timedelta
from collections import defaultdict

api_key = 'mzrsaV5E9ZwVPQ2biOQGyYansI7Ukh5p'
api_key = 'CGeuqVOhGnPEhw8fZaxDDpSXMklkLVJo'


In [21]:
def print_json(json_data):
    print(json.dumps(json_data, indent=2))

def write_jsonl_file(data, filename):
    # Write data to a JSONL file
    with open(filename, 'a', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')


In [22]:
def return_response(url) :
    response = requests.get(url)
    return response.json()

def generate_date_range(start_date, end_date):
    # Generate a date range between start_date and end_date
    current_date = start_date
    while current_date <= end_date:
        yield current_date
        current_date += timedelta(weeks=1)


# Getting All Lists

In [7]:
# output_filename = 'all_lists.json'
# with open(output_filename, 'w', encoding='utf-8') as file:
#     url = f'https://api.nytimes.com/svc/books/v3/lists/names.json?api-key={api_key}' # all lists
#     response = return_response(url)

#     json.dump(response, file, indent=4)
#     print(f'All lists saved to {output_filename}.')


All lists saved to all_lists.json.


# Getting All Weeks

In [28]:
start_date_str = '2011-02-13'
start_date_str = '2017-01-01' # API breakpoint
end_date_str = '2023-12-10'

start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

all_responses = {}
failed_week = []

output_filename = 'all_weeks_2.json'
with open(output_filename, 'w', encoding='utf-8') as file:
    for current_date in generate_date_range(start_date, end_date):
        time.sleep(12) #
        week = current_date.strftime('%Y-%m-%d')
        url = f'https://api.nytimes.com/svc/books/v3/lists/{week}/combined-print-and-e-book-fiction.json?api-key={api_key}'
        response = return_response(url)

        if ('status' in response and response['status']=='OK'):
            book = response['results']['books'][0]
            book = {
                'date': response['results']['published_date'],
                'rank': book['rank'],
                'title': book['title'],
                'author': book['author'],
                'primary_isbn10': book['primary_isbn10'],
                'dagger': book['dagger'],
                'publisher': book['publisher'],
                'weeks_on_list': book['weeks_on_list']
            }
            month = book['date'].split("-")[0]+'-'+book['date'].split("-")[1]
            if month in all_responses:
                all_responses[month].append(book)
            else:
                all_responses[month] = [book]

            print(f'Response for week {week} added to the list.')

        else:
            print(week)
            failed_week.append(week)

    json.dump(all_responses, file, indent=2)


Response for week 2017-01-01 added to the list.
Response for week 2017-01-08 added to the list.
Response for week 2017-01-15 added to the list.
Response for week 2017-01-22 added to the list.
Response for week 2017-01-29 added to the list.
Response for week 2017-02-05 added to the list.
Response for week 2017-02-12 added to the list.
Response for week 2017-02-19 added to the list.
Response for week 2017-02-26 added to the list.
Response for week 2017-03-05 added to the list.
Response for week 2017-03-12 added to the list.
Response for week 2017-03-19 added to the list.
Response for week 2017-03-26 added to the list.
Response for week 2017-04-02 added to the list.
Response for week 2017-04-09 added to the list.
Response for week 2017-04-16 added to the list.
Response for week 2017-04-23 added to the list.
Response for week 2017-04-30 added to the list.
Response for week 2017-05-07 added to the list.
Response for week 2017-05-14 added to the list.
Response for week 2017-05-21 added to th

# Parsing data into months

In [ ]:
# from collections import Counter

# all_responses_month = {}

# input_filename = 'all_weeks.json'
# output_filename = 'all_months.json'
# with open(output_filename, 'w', encoding='utf-8') as file:
#     with open(input_filename, 'r', encoding='utf-8') as read_file:
#         data = json.load(read_file)

#         for month in data.keys():
#             book_list = data[month]

#             titles = [book['title'] for book in book_list]
#             title_counts = Counter(titles)
#             most_common_title = title_counts.most_common(1)[0][0]

#             most_common_book = next(book for book in book_list if book['title'] == most_common_title)
#             all_responses_month[month] = most_common_book 
            
#     json.dump(all_responses_month, file, indent=2)


In [31]:
import csv
from collections import Counter
import json

all_responses_month = {}

input_filename = 'all_weeks.json'
output_filename = 'p3js/all_months.csv'

with open(output_filename, 'w', encoding='utf-8', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    header = ['rank', 'title', 'author','primary_isbn10','dagger','publisher','weeks_on_list']
    csv_writer.writerow(['month']+header)  # Add other column headers as needed

    with open(input_filename, 'r', encoding='utf-8') as read_file:
        data = json.load(read_file)

        for month in data.keys():
            book_list = data[month]

            titles = [book['title'] for book in book_list]
            title_counts = Counter(titles)
            most_common_title = title_counts.most_common(1)[0][0]

            most_common_book = next(book for book in book_list if book['title'] == most_common_title)
            all_responses_month[month] = most_common_book

            row = [most_common_book[key] for key in header]
            csv_writer.writerow([month]+row)  # Add other column values as needed
